# HR ATTRITION ANALYSIS

In this analysis, we explore attrition and the main causes while employees leave the company. To help with our analysis we will be asking possible questions that the management might have regarding our dataset. A prediction model will also help us determine what are the main categories that contribute why employees leave their jobs. At the end of this notebook, a recommendation will be provide on how we can tackle this challenge.

OUTLINE:

PART 1: DATA AND LIBRARY LOADING

PART 2: DATA UNDERSTANDING

PART 3: GRAPH THE DATASET

PART 4: DATA PREPARATION

PART 5: MODELING and EVALUATION

PART 6: DEPLOYMENT

## PART 1: DATA AND LIBRARY LOADING

In [2]:
# This is to remove the warning messages while the codes will run.
# Do this after the final draft
shh <- suppressPackageStartupMessages
install.packages("ggcorrplot")

# Load the appropriate libararies
library (ggplot2)
library(caret)
library(randomForest)
library(ggthemes)
library(pROC)
library(rpart)
library(plyr)
library(RColorBrewer)
library(ggcorrplot)
library(corrplot)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
corrplot 0.84 loaded


In [3]:
# Load the CSV file with employee data
hr_data <- read.csv('http://bit.ly/hr_data_2')

## PART 2: DATA UNDERSTANDING

In [27]:
# This code will help us identify the different fields we have in the dataset

str(hr_data)

'data.frame':	1470 obs. of  25 variables:
 $ Attrition              : Factor w/ 2 levels "No","Yes": 2 1 2 1 1 1 1 1 1 1 ...
 $ EmployeeNumber         : int  1 2 4 5 7 8 10 11 12 13 ...
 $ Age                    : int  41 49 37 33 27 32 59 30 38 36 ...
 $ Gender                 : Factor w/ 2 levels "Female","Male": 1 2 2 1 2 2 1 2 2 2 ...
 $ MaritalStatus          : Factor w/ 3 levels "Divorced","Married",..: 3 2 3 2 2 3 2 1 3 2 ...
 $ Department             : Factor w/ 3 levels "Human Resources",..: 3 2 2 2 2 2 2 2 2 2 ...
 $ JobRole                : Factor w/ 9 levels "Healthcare Representative",..: 8 7 3 7 3 3 3 3 5 1 ...
 $ MonthlyIncome          : int  5993 5130 2090 2909 3468 3068 2670 2693 9526 5237 ...
 $ OverTime               : Factor w/ 2 levels "No","Yes": 2 1 2 2 1 1 2 1 1 1 ...
 $ EducationField         : Factor w/ 6 levels "Human Resources",..: 2 2 5 2 4 2 4 2 2 4 ...
 $ BusinessTravel         : Factor w/ 3 levels "Non-Travel","Travel_Frequently",..: 3 2 3 2 3 2 3 3 2 3 

In [32]:
# This code is to help us see the first few records in our dataset
head(hr_data)

Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,PerformanceRating,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
<fct>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Yes,1,41,Female,Single,Sales,Sales Executive,5993,Yes,Life Sciences,...,3,11,0,0,8,8,6,4,0,5
No,2,49,Male,Married,Research & Development,Research Scientist,5130,No,Life Sciences,...,4,23,1,3,10,1,10,7,1,7
Yes,4,37,Male,Single,Research & Development,Laboratory Technician,2090,Yes,Other,...,3,15,0,3,7,6,0,0,0,0
No,5,33,Female,Married,Research & Development,Research Scientist,2909,Yes,Life Sciences,...,3,11,0,3,8,1,8,7,3,0
No,7,27,Male,Married,Research & Development,Laboratory Technician,3468,No,Medical,...,3,12,1,3,6,9,2,2,2,2
No,8,32,Male,Single,Research & Development,Laboratory Technician,3068,No,Life Sciences,...,3,13,0,2,8,0,7,7,3,6


In [4]:
# This code calculates the descriptive statistics of each field in our dataset
summary(hr_data)

 Attrition  EmployeeNumber        Age           Gender     MaritalStatus
 No :1233   Min.   :   1.0   Min.   :18.00   Female:588   Divorced:327  
 Yes: 237   1st Qu.: 491.2   1st Qu.:30.00   Male  :882   Married :673  
            Median :1020.5   Median :36.00                Single  :470  
            Mean   :1024.9   Mean   :36.92                              
            3rd Qu.:1555.8   3rd Qu.:43.00                              
            Max.   :2068.0   Max.   :60.00                              
                                                                        
                  Department                       JobRole    MonthlyIncome  
 Human Resources       : 63   Sales Executive          :326   Min.   : 1009  
 Research & Development:961   Research Scientist       :292   1st Qu.: 2911  
 Sales                 :446   Laboratory Technician    :259   Median : 4919  
                              Manufacturing Director   :145   Mean   : 6503  
                          

##### This line has multiple codes because I'd have to adjust Attrition to an integer so that the corrplot can read my dataset. After that, I'd have to put it back into a factor so that it won't mess up the rest of the analysis

In [4]:
# This code allows us to see the correlation of each fields to each other

hr_data$Attrition <- as.integer(hr_data$Attrition)

M <-cor(hr_data[c(1,3,8,12:25)])
corrplot(M, method="color")

hr_data$Attrition <- factor(hr_data$Attrition)

## PART 3: GRAPH THE DATASET

Prior to building a predictive model, we graph the dataset to better understand attrition by answering several questions that were posted by management

### QUESTION #1: "Does one gender tend to quit more frequently than the other?"

Based on the first graph for this question, it seems that the males have a higher number of attrition compare to females. We might say that more males are quitting the company compared to females. However, this might not be accurate since there are more males than females. To confirm this claim, we made a second graph that shows the rate of attrition according to the total number of employees. In this new graph, we can see that there is not much of a difference of attrition in gender. Males are only a few percentages higher than females when it comes to quitting in the company.   

In [5]:
# Create a bar plot that counts employees by gender and attrition.

ggplot(hr_data,aes(x = Gender,)) +
    geom_bar(aes(fill=factor(Attrition)))+
    ggtitle("Attrition # by Gender") + 
    ylab("# of Employees") + labs(fill = "Quit?") + scale_fill_manual(values=c("#999999", "#E69F00"))

In [6]:
ggplot(hr_data, aes(x = Gender, fill = Attrition)) + geom_bar(position = 'fill')+
    ggtitle("Attrition % by Gender")  + scale_fill_manual(values=c("#999999", "#E69F00")) +
    ylab("Percentage") +
    scale_y_continuous(labels = function(x) paste0(x*100, "%")) + labs(fill = "Quit?")

### QUESTION #2: "Does age appear to make a difference? Are our younger workers less committed to our company?"

Based on our findings in the graph below, it seems that most of the people who quit their jobs are ages between 25-35 yr olds. However, this can be different again because of the total number of people in those age group. So, to see avoid this confusion, we are going to take a look at the rate of people quitting in each age group. The second graph shows the rate and it looks like the younger worker have a high rate of attrition. However, when younger workers leave the company, it doesn't necessarily mean they are less committed. It is a trend for millinials leaving the workforce due to desire of growth, change, improvement, and opportunities. So they basically look for these things in another firm.

In [8]:
# Create a that counts employees by age and attrition

ggplot(hr_data, aes(x = Age, fill = Attrition)) + ggtitle("Attrition by Age Group") +
    geom_histogram(binwidth = 2) + labs(fill = "Quit?") +
    scale_fill_manual(values=c("#999999", "#E69F00")) + ylab("# of Employees")

In [10]:
ggplot(hr_data, aes(x = Age, fill = Attrition)) +
    geom_histogram(binwidth = 2, position = 'fill') + labs(fill = "Quit?") +
    ggtitle("Attrition % by Age Group") +
    scale_fill_manual(values=c("#999999", "#E69F00")) + ylab("Percentage") + 
    scale_y_continuous(labels = scales::percent)

### QUESTION #3: "Are we working our employees too hard? Is our overtime requirement for some employees driving them away?"

Based on the graph below, there is a fraction of our workers who are doing overtime. It seems that those who are doing overtime have a higher rate of quitting the company. To validate this, we made another graph to show the rate of employees leaving the company. Of those doing overtime, over 25% of them leave the company. Overtime can be a big problem especially when the employees are not receiving any incentives when they go over their scheduled time to work. This analysis will provide some information on what other companies are doing already and also add my own recommendation regarding this problem.

In [11]:
# 
ggplot(hr_data, aes(x = OverTime ,fill=Attrition)) + 
    geom_bar() + 
    ggtitle("Over Time") + labs(fill = "Quit?") + ylab("# of Employees") +
    scale_fill_manual(values=c("#999999", "#E69F00")) + 
    theme(plot.title = element_text(size =10),
    axis.text.x = element_text(size =7,angle = 45, hjust = 1),
    axis.title.x=element_blank()) + scale_y_continuous(limits = c(0, 1100))

In [39]:
ggplot(hr_data, aes(x = OverTime ,fill=Attrition)) + 
    geom_bar(position='fill') + 
    ggtitle("% of Over Time") + labs(fill = "Quit?") + ylab("# of Employees") +
    scale_fill_manual(values=c("#999999", "#E69F00")) + 
    theme(plot.title = element_text(size =10),
    axis.text.x = element_text(size =7,angle = 45, hjust = 1),
    axis.title.x=element_blank()) +
    scale_y_continuous(labels = function(x) paste0(x*100, "%"))

### QUESTION #4: "Is this about money? Are we not paying employees enough? Were the raises not strong enough? Is that why they leave?"

Based on the graph below, we can see that those employees who have a monthly income of $5000 and below have the most employees leaving. However, a low monthly income may not be the only reason why employees leave. Additional graphing of the dataset will help us understand it more. In our second paragraph, it seems that those who have a low percentage of salary hike to tend to leave the company more. 

In [17]:
#
ggplot(hr_data,aes(MonthlyIncome,fill=Attrition))+
    geom_density() + labs(fill = "Quit?") + 
    scale_fill_manual(values=c("#999999", "#E69F00")) + 
    xlab("Monthly Income")

In [61]:
ggplot(hr_data,aes(x = PercentSalaryHike, fill = Attrition)) + 
    geom_density()+ labs(fill = "Quit?") +
    scale_fill_manual(values=c("#999999", "#E69F00")) + 
    xlab("% Salary Hike") + ggtitle("Percentage Salary Hike") 
   

### QUESTION #5: "Our concern is that the highest performing employees are leaving at a higher rate. Is that true?"

Based

In [21]:
#
ggplot(hr_data,aes(PerformanceRating ,fill=Attrition))+
    geom_density() + labs(fill = "Quit?") +
    scale_fill_manual(values=c("#999999", "#E69F00")) + 
    xlab("Performance Rating") 

In [45]:
#
ggplot(hr_data,aes(Attrition ,PerformanceRating))+
    geom_count()



## PART 4: DATA PREPARATION

Explanation here

### First, we have to check if our dataset have some missing values

If we have missing values in our dataset, it might throw off our prediction.

In [5]:
# Sum the number of null or na in hr_data
sum(is.na(hr_data))
which(is.na.data.frame(hr_data))

[1] 0

integer(0)

### Second, we need to divide our dataset into our train and test data

Separating data into training and testing sets is an important part of evaluating data mining models. Typically, when you separate a data set into a training set and testing set, most of the data is used for training, and a smaller portion of the data is used for testing. R randomly samples the data to help ensure that the testing and training sets are similar. By using similar data for training and testing, you can minimize the effects of data discrepancies and better understand the characteristics of the model.

After a model has been processed by using the training set, you test the model by making predictions against the test set. Because the data in the testing set already contains known values for the attribute that you want to predict, it is easy to determine whether the model's guesses are correct.

In [6]:
# To split the data set
# This helps us in the random generation numbers
set.seed(3456)
trainIndex <- createDataPartition(hr_data$Attrition, p=.8, 
                                  list=FALSE, times = 1)

hr_train <- hr_data[trainIndex,]
hr_test  <- hr_data[-trainIndex,]

# This is to check if the splitting of data worked
head(hr_train)
head(hr_test)

,Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,PerformanceRating,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
,<fct>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,2,1,41,Female,Single,Sales,Sales Executive,5993,Yes,Life Sciences,...,3,11,0,0,8,8,6,4,0,5
3,2,4,37,Male,Single,Research & Development,Laboratory Technician,2090,Yes,Other,...,3,15,0,3,7,6,0,0,0,0
4,1,5,33,Female,Married,Research & Development,Research Scientist,2909,Yes,Life Sciences,...,3,11,0,3,8,1,8,7,3,0
5,1,7,27,Male,Married,Research & Development,Laboratory Technician,3468,No,Medical,...,3,12,1,3,6,9,2,2,2,2
6,1,8,32,Male,Single,Research & Development,Laboratory Technician,3068,No,Life Sciences,...,3,13,0,2,8,0,7,7,3,6
7,1,10,59,Female,Married,Research & Development,Laboratory Technician,2670,Yes,Medical,...,4,20,3,3,12,4,1,0,0,0


,Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,PerformanceRating,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
,<fct>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
2,1,2,49,Male,Married,Research & Development,Research Scientist,5130,No,Life Sciences,...,4,23,1,3,10,1,10,7,1,7
14,1,18,34,Male,Divorced,Research & Development,Laboratory Technician,2661,No,Medical,...,3,11,1,2,3,0,2,2,1,2
15,2,19,28,Male,Single,Research & Development,Laboratory Technician,2028,Yes,Life Sciences,...,3,14,0,4,6,5,4,2,0,3
16,1,20,29,Female,Divorced,Research & Development,Manufacturing Director,9980,No,Life Sciences,...,3,11,1,1,10,1,10,9,8,8
20,1,24,38,Male,Single,Research & Development,Research Scientist,3944,Yes,Life Sciences,...,3,11,0,3,6,5,3,2,1,2
23,1,28,34,Female,Single,Research & Development,Research Director,11994,No,Life Sciences,...,3,11,0,4,13,0,12,6,2,11


### Third, converting integers into factors

This convertion of intergers into factors will help our model work

In [7]:
# Converting the categories to factors
hr_train$StockOptionLevel <- factor(hr_train$StockOptionLevel)
hr_test$StockOptionLevel <- factor(hr_test$StockOptionLevel)

## PART 5: MODELING and EVALUATION

This section will train, test and evaluate our dataset. We will have Logistic Regression 
This model will follow the outlined steps below.

Outline

STEP 1: Version 1 - using all the variable

STEP 2: Make Predictions

STEP 3: Evaluate using ROC-Curve

STEP 4: Narrow down using varImp

STEP 5: Version 2 - narrowed version

STEP 6: Make Predictions

STEP 7: Evaluate using ROC-Curve

### Logistic Regression Model

Logistic regression is yet another technique borrowed by machine learning from the field of statistics. It's a powerful statistical way of modeling a binomial outcome with one or more explanatory variables. It measures the relationship between the categorical dependent variable and one or more independent variables by estimating probabilities using a logistic function, which is the cumulative logistic distribution.

#### STEP 1

Version 1: USING ALL VARIABLES

In [8]:
# To create a prediciton model using glm (logistic Regression)
hr_model_logreg <- glm(Attrition ~ ., data = hr_train, family = binomial)

# To display the prediction model created above
summary(hr_model_logreg)


Call:
glm(formula = Attrition ~ ., family = binomial, data = hr_train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8593  -0.4790  -0.2508  -0.0915   3.5194  

Coefficients:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -1.243e+01  6.688e+02  -0.019 0.985167    
EmployeeNumber                   -1.029e-04  1.656e-04  -0.621 0.534281    
Age                              -2.924e-02  1.456e-02  -2.008 0.044613 *  
GenderMale                        3.272e-01  2.041e-01   1.603 0.108893    
MaritalStatusMarried             -3.182e-02  3.057e-01  -0.104 0.917105    
MaritalStatusSingle              -1.568e-02  4.214e-01  -0.037 0.970314    
DepartmentResearch & Development  1.326e+01  6.688e+02   0.020 0.984183    
DepartmentSales                   1.330e+01  6.688e+02   0.020 0.984135    
JobRoleHuman Resources            1.412e+01  6.688e+02   0.021 0.983159    
JobRoleLaboratory Technician      1.330

#### STEP 2

Make Predictions

In [10]:
hr_test$Attrition_Prediction_lr_01 <- predict(hr_model_logreg, 
                newdata = hr_test, type = "response")

print(head(hr_test[c("Attrition", "Attrition_Prediction_lr_01")]))

   Attrition Attrition_Prediction_lr_01
2          1                 0.02113101
14         1                 0.04412419
15         2                 0.88957971
16         1                 0.02159513
20         1                 0.40566315
23         1                 0.00265202


#### STEP 3 

Evaluate using ROC-Curve

In [11]:
ROC_lr_01 <- roc(hr_test$Attrition, hr_test$Attrition_Prediction_lr_01)

plot(ROC_lr_01)
auc(ROC_lr_01)

Setting levels: control = 1, case = 2
Setting direction: controls < cases


Area under the curve: 0.8031

#### STEP 4

NARROW DOWN USING varImp

In [12]:
# This code will evaluate the variable of importance for each of our categories
# It will also sort it from highest to lowest
# This will add up to a hundred because it is relative importance

importance <- as.data.frame(varImp(hr_model_logreg))
importance <- data.frame(Importance = importance$Overall,
                        Field = rownames(importance))
importance[order(-importance$Importance),]

,Importance,Field
,<dbl>,<fct>
17,9.17524675,OverTimeYes
36,5.03113160,NumCompaniesWorked
31,4.51652887,StockOptionLevel1
28,4.32037957,JobSatisfaction
23,4.08680824,BusinessTravelTravel_Frequently
39,3.87151020,YearsSinceLastPromotion
27,3.66179645,JobInvolvement
32,3.47591344,StockOptionLevel2
25,3.29966234,DistanceFromHome


#### STEP 5

Version 2: NARROWED VERSION

In [13]:
# The best prediction model according to variable of improtance

# MY THOUGHTS
#I tried JobRole, Gender and Age and it doesn't improve the model

hr_model_logreg_01 <- glm(Attrition ~ OverTime + NumCompaniesWorked +
    JobSatisfaction + BusinessTravel + YearsSinceLastPromotion +
    DistanceFromHome + JobInvolvement + YearsWithCurrManager +
    YearsInCurrentRole + TrainingTimesLastYear + Age + MaritalStatus +
    JobRole + StockOptionLevel + TotalWorkingYears + Gender
    , data = hr_train, family = binomial)

# To display the prediction model created above
summary(hr_model_logreg_01)


Call:
glm(formula = Attrition ~ OverTime + NumCompaniesWorked + JobSatisfaction + 
    BusinessTravel + YearsSinceLastPromotion + DistanceFromHome + 
    JobInvolvement + YearsWithCurrManager + YearsInCurrentRole + 
    TrainingTimesLastYear + Age + MaritalStatus + JobRole + StockOptionLevel + 
    TotalWorkingYears + Gender, family = binomial, data = hr_train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8141  -0.4975  -0.2658  -0.1028   3.4608  

Coefficients:
                                Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      0.26160    0.92337   0.283 0.776943    
OverTimeYes                      1.89000    0.21064   8.973  < 2e-16 ***
NumCompaniesWorked               0.18857    0.04058   4.647 3.37e-06 ***
JobSatisfaction                 -0.38246    0.08716  -4.388 1.14e-05 ***
BusinessTravelTravel_Frequently  1.71732    0.42600   4.031 5.55e-05 ***
BusinessTravelTravel_Rarely      0.80199    0.39279   2.042 0.041177

#### STEP 6

Make Predictions

In [20]:
# This is to the include the model of predicting attrition in our test data
hr_test$Attrition_Prediction_lr_02 <- predict(hr_model_logreg_01, 
                newdata = hr_test, type = "response")

# Additional changes were made according to notes from class
print(head(hr_test[c("Attrition", "Attrition_Prediction_lr_02")]))

str(hr_test)

   Attrition Attrition_Prediction_lr_02
2          1                0.024895803
14         1                0.056781438
15         2                0.847902873
16         1                0.032824872
20         1                0.364690170
23         1                0.003264698
'data.frame':	293 obs. of  27 variables:
 $ Attrition                 : Factor w/ 2 levels "1","2": 1 1 2 1 1 1 1 1 1 1 ...
 $ EmployeeNumber            : int  2 18 19 20 24 28 30 40 46 51 ...
 $ Age                       : int  49 34 28 29 38 34 21 44 43 36 ...
 $ Gender                    : Factor w/ 2 levels "Female","Male": 2 2 2 1 2 1 2 2 1 1 ...
 $ MaritalStatus             : Factor w/ 3 levels "Divorced","Married",..: 2 1 3 1 3 3 3 2 1 2 ...
 $ Department                : Factor w/ 3 levels "Human Resources",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ JobRole                   : Factor w/ 9 levels "Healthcare Representative",..: 7 3 3 5 7 6 7 1 7 7 ...
 $ MonthlyIncome             : int  5130 2661 2028 9980 3944 1199

#### STEP 7

Evaluate using ROC-Curve

In [ ]:
# The purpose of this is to test and validate if our model is reliable
# Making your Area Under the ROC Curve
# 1st ROC Curve Include all the variables
# 2nd ROC Curve only includes the narrower variables

In [17]:
ROC_lr_02 <- roc(hr_test$Attrition, hr_test$Attrition_Prediction_lr_02)
plot(ROC_lr_02)
auc(ROC_lr_02)



# It is going to be a value under 1. This is evaluating classifications
# What does this ROC-curve used for?
# Confusion matrix - See the video for more info
# The closer the curve to the left (1.0 for sensitivity & 0.0 for specificity)
# The higher the value the better the prediction is

Setting levels: control = 1, case = 2
Setting direction: controls < cases


Area under the curve: 0.8139

ERROR: Error in roc.default(hr_test$Attrition, hr_test$hr_test$Attrition_Prediction_lr_01): No valid data provided.


In [19]:
# Evaluate the logistic regression models.

# Generate ROC curves for both version of the L.R. model.
ROC_lr_1 <- roc(hr_test$Attrition, hr_test$Attrition_Prediction_lr_01)
ROC_lr_2 <- roc(hr_test$Attrition, hr_test$Attrition_Prediction_lr_02)

# Print the AUC for both versions of the model
paste("AUC for Logistic Regression Model (all variables):", round(auc(ROC_lr_1),2), "(red)")
paste("AUC for Logistic Regression Model (limited variables):", round(auc(ROC_lr_2),2), "(blue)")

# Plot the ROC curves.
plot.roc(ROC_lr_1, col="red")
lines.roc(ROC_lr_2, col="blue")

Setting levels: control = 1, case = 2
Setting direction: controls < cases
Setting levels: control = 1, case = 2
Setting direction: controls < cases


[1] "AUC for Logistic Regression Model (all variables): 0.8 (red)"

[1] "AUC for Logistic Regression Model (limited variables): 0.81 (blue)"

## PART 6: DEPLOYMENT

Short explanation here

In [22]:
# Write predictions to the test file.
hr_test_abbreviated <- hr_test[,c(2,26)]
hr_test_abbreviated <- hr_test[,c("EmployeeNumber","Attrition_Prediction_lr_02")]
head(hr_test_abbreviated)
write.csv(hr_test_abbreviated,"hr_predictions.csv",row.names=FALSE)



,EmployeeNumber,Attrition_Prediction_lr_02
,<int>,<dbl>
2,2,0.024895803
14,18,0.056781438
15,19,0.847902873
16,20,0.032824872
20,24,0.364690170
23,28,0.003264698


# Recommendation for HR Attrition Analysis

1. Employee over time is one factor why most employee leave the company. We recommend that they be offered a higher overtime pay to keep them motivated or hire more employees to avoid overtime.

2. Distance from the workplace is also one factor that makes employee life difficult. We recommend that a shuttle service system be established for those who are far from the workplace to help them lessen transportation expense.

Notes from class: We can recommend to have them work in their home once or twice a week

3. Job Involvement refers to the pyschological and emotional extent to which someone participates in his/her work, profession, and company. We noticed that the company have a low job involvement score which also contributes to employees losing interest in the workplace and leaving. We recommend to have rewards system and engaging activities be set up during working hours to help employees be engaged and be able to develop strong relationship/friendship with each other. This will encourage them to perform better.